In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv("race.csv",encoding='utf-8')
data = data.dropna(subset=['record'])
# data = data.dropna() # 이거로 하면 cost 5자리수 나옴


y = data['record']
X = data.loc[:, ["code","lane","sex","age","jockey_w","jockey","dandivi","yeondivi"
                    ,"s1f","c3","c4","g3f","g1f","cure_in_1m",
                 "weight_diff","raw_weight","weight_added","prev1_velo","horse_level_num"]]

#print(X.shape)

# sex는 dummy 함수 처리
sex_mod = pd.get_dummies(X['sex'])
X = X.join(sex_mod)
X = X.drop(columns=['sex'])
#print(X.columns)
X = X.as_matrix()

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)

# 설정하는 부분
learning_rate = 0.001
training_epochs = 10
batch_size = 16


train_size :  48596
size_x :  21


In [86]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)



W1 = tf.get_variable(shape=[size_x, 256], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)


W2 = tf.get_variable(shape=[256, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)


W3 = tf.get_variable(shape=[64, 1], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer2, W3) + b3

cost = tf.reduce_mean(tf.square(hypothesis - Y))
#cost = tf.losses.mean_squared_error(labels=Y, predictions=hypothesis)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        batch_xs = X_train[i*16:(i+1)*16]
        batch_ys = y_train[i*16:(i+1)*16].reshape(16,1)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, train], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Epoch: 0001 cost = nan
Epoch: 0002 cost = nan
Epoch: 0003 cost = nan
Epoch: 0004 cost = nan
Epoch: 0005 cost = nan
Epoch: 0006 cost = nan
Epoch: 0007 cost = nan
Epoch: 0008 cost = nan
Epoch: 0009 cost = nan
Epoch: 0010 cost = nan
